In [ ]:
import torch
from torch.utils.data import DataLoader
from torchmetrics import JaccardIndex
import segmentation_models_pytorch as smp

# CẤU HÌNH LẠI CÁC BIẾN CẦN THIẾT 
# Đảm bảo Device và Model đã có
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(DEVICE) # Đưa model lên GPU

# Đường dẫn (Kiểm tra kỹ xem folder này có tồn tại không)
train_dir = '/kaggle/input/deepglobe-land-cover-classification-dataset/train' 
val_dir   = '/kaggle/input/deepglobe-land-cover-classification-dataset/test'  

#SETUP METRIC (Dùng lại JaccardIndex cho đồng bộ với lúc Train) 
loss = smp.losses.DiceLoss(mode='binary')
loss.__name__ = 'dice_loss'

# Dùng TorchMetrics (ổn định hơn smp.utils.metrics)
iou_metric = JaccardIndex(task="binary", threshold=0.5).to(DEVICE)
iou_metric.__name__ = 'iou_score'
metrics = [iou_metric]

#  TẠO DATASET VÀ LOADER (Đã sửa lỗi thiếu masks_dir) 

# Tập Train
train_dataset = CloudDataset(
    images_dir=train_dir,      # Truyền rõ tên tham số
    masks_dir=train_dir,       # <--- QUAN TRỌNG: Phải có dòng này
    preprocessing=preprocessing_fn,
    has_masks=True
)

# Tập Valid (Test)
valid_dataset = CloudDataset(
    images_dir=val_dir,        # Truyền rõ tên tham số
    masks_dir=val_dir,         # <--- QUAN TRỌNG: Phải có dòng này
    preprocessing=preprocessing_fn,
    has_masks=True
)

# Loader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=0)

# CHẠY CHẤM ĐIỂM (VALIDATION) 
# Tạo Valid Epoch Runner
valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

print("\n--- ĐANG CHẤM ĐIỂM TRÊN TẬP TEST ---")
# Chạy chấm điểm
# Lưu ý: Nếu tập test không có file mask (đuôi _mask.png) thì đoạn này sẽ lỗi.
# Hãy đảm bảo folder 'test' có đầy đủ cả ảnh _sat.jpg và _mask.png
try:
    valid_logs = valid_epoch.run(valid_loader)
    print("\n=== KẾT QUẢ CUỐI CÙNG ===")
    print(f"Dice Loss: {valid_logs['dice_loss']:.4f}")
    print(f"IoU Score: {valid_logs['iou_score']:.4f}")
except Exception as e:
    print(f"\nLỖI KHI CHẠY: {e}")
    print("Gợi ý: Kiểm tra lại xem folder '/test' có chứa file '_mask.png' không?")
